# Sample Notebook



Welcome! This notebook is an overview of the XNAT/Jupyter integegration. Note that this isn’t in an XNAT release yet but will be coming in the next couple of months. You’re welcome to start previewing the feature in the meantime.

## Imports

In [1]:
import os
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from io import StringIO

Ideally packages should be bundled with the image. But you can install packages if needed.

In [ ]:
pip install xnat

## Environmental Variables

XNAT adds environmental variables to the single-user Jupyter containers to:

  1. Identify the XNAT context in which Jupyter was started
  2. Connect back to XNAT's REST API from inside the container
  
Let's print out these environmental variables:

In [ ]:
for name, value in os.environ.items():
    if 'XNAT' in name:
        print("{0}: {1}".format(name, value))

The XNAT_HOST, XNAT_USER and XNAT_PASS environmental variables can be used to make a connect back to XNAT's REST API

JupyterHub also adds some environmental variables you might find helpful

In [ ]:
for name, value in os.environ.items():
    if 'JUPYTER' in name:
        print("{0}: {1}".format(name, value))

## Navigating the file system

When you launch Jupyter from XNAT, under the hood XNAT is gathering a list of mount points that will be used for your single-user Jupyter container. JupyterHub is configured to use Docker bind mounts for exposing parts of the XNAT data archive to a running single-user Jupyter container. The mounts vary depending on the context in which you launched Jupyter. The Jupyter container will also be mounted with a workspace diretory to store your notebooks. Notebooks stored here will persist in between Jupyter sessions. Any data or notebooks stored outside of this workspace directory will be lost when stopping your Jupyter container.

Your workspace directory is located at /workspace/{username}. Here is a helpful env variable

In [ ]:
os.environ['JUPYTERHUB_ROOT_DIR']

Next we will walk the file system to see the XNAT data that's been mounted.

In [ ]:
i = 0
stop = 2
for root, dirs, files in os.walk(os.environ['XNAT_DATA'], topdown=False):
   for name in files:
      print(os.path.join(root, name))
   for name in dirs:
      print(os.path.join(root, name))
        
   i = i + 1
   if i > stop:
       break

## Using the XNAT REST API

You can use environmental variables to connect to the [XNAT REST API](https://wiki.xnat.org/display/XAPI/XNAT+REST+API+Directory). **XNAT_HOST** is your XNAT url. **XNAT_USER** and **XNAT_PASS** is an alias token for your XNAT user account.

This is a simple example of connecting to the XNAT REST API. In general you're probably better off using **XNATpy** or **pyxnat** instead of working directly with with the XNAT REST API.

In [7]:
host = os.environ['XNAT_HOST']
username = os.environ['XNAT_USER']
password = os.environ['XNAT_PASS']

In [8]:
r=requests.get(f'{host}/data/projects',
               auth=HTTPBasicAuth(username, password))
if not r.ok:
        print(f'Failed to get projects')

In [9]:
projects = r.json()['ResultSet']['Result']

## Working with Stored Searches
After saving a stored search in XNAT you may want to convert that to a panadas DataFrame.

This is example starts with a stored search of a CT scan joined with the Subject table. You'll need to create a similar stored search in XNAT before executing.

Start by calling the [Stored Search API](https://wiki.xnat.org/display/XAPI/Stored+Search+API) to get the list of stored searches.

In [11]:
host = os.environ['XNAT_HOST']
username = os.environ['XNAT_USER']
password = os.environ['XNAT_PASS']

In [12]:
r = requests.get(f'{host}/data/search/saved',
                 auth=HTTPBasicAuth(username, password))

if not r.ok:
        print(f'Failed to get searches')

In [13]:
searches = r.json()['ResultSet']['Result']

Get the search id for our CT scan stored search

In [14]:
ct_scan_search_id = 'xs1668708947057'
r = requests.get(f'{host}/data/search/saved/{ct_scan_search_id}/results?format=csv&guiStyle=true', 
                 auth=HTTPBasicAuth(username, password))
df = pd.read_csv(StringIO(r.text), sep=",")

Lets add the file path of each scan to the dataframe.

In [15]:
df = df.assign(path="/data/projects/" + df["project"] + "/experiments/" + df["Session Label"] + "/SCANS/" + df["id"].astype(str) + "/DICOM")

Filter out unused columns. This could/should be done in XNAT in the stored search directly.

In [16]:
df = df[["project", "Session Label", "id", "series_description", "GENDER", "Age", "bodyPartExamined", "path"]]

Lets select only abdomen arterial contrasts scans.

In [17]:
df = df.loc[(df["bodyPartExamined"] == "ABDOMEN") & (df["series_description"] == "arterial")]
df

,project,Session Label,id,series_description,GENDER,Age,bodyPartExamined,path
4,C4KC-KiTS,KiTS-00023_CT_1,4,arterial,male,49,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0002...
6,C4KC-KiTS,KiTS-00030_CT_1,4,arterial,female,64,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0003...
7,C4KC-KiTS,KiTS-00014_CT_1,3,arterial,female,67,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0001...
8,C4KC-KiTS,KiTS-00046_CT_1,2,arterial,male,38,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0004...
9,C4KC-KiTS,KiTS-00021_CT_1,6,arterial,female,73,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0002...
12,C4KC-KiTS,KiTS-00010_CT_1,4,arterial,female,71,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0001...
14,C4KC-KiTS,KiTS-00033_CT_1,3,arterial,female,69,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0003...
16,C4KC-KiTS,KiTS-00000_CT_1,7,arterial,male,49,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0000...
21,C4KC-KiTS,KiTS-00017_CT_1,2,arterial,female,44,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0001...
22,C4KC-KiTS,KiTS-00043_CT_1,3,arterial,male,62,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0004...


You can then further refine your selection say based on gender and age.

In [18]:
df.loc[(df["GENDER"] == "male") & (df["Age"] < 50)]

,project,Session Label,id,series_description,GENDER,Age,bodyPartExamined,path
4,C4KC-KiTS,KiTS-00023_CT_1,4,arterial,male,49,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0002...
8,C4KC-KiTS,KiTS-00046_CT_1,2,arterial,male,38,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0004...
16,C4KC-KiTS,KiTS-00000_CT_1,7,arterial,male,49,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0000...
49,C4KC-KiTS,KiTS-00034_CT_1,3,arterial,male,49,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0003...
68,C4KC-KiTS,KiTS-00005_CT_1,6,arterial,male,35,ABDOMEN,/data/projects/C4KC-KiTS/experiments/KiTS-0000...
